This notebook take a stock and creates a stacked six time series graphs:

1. Stock closing price
2. Volume
3. RSI
4. Stoch
5. MACD
6. Mass Index

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Create data frame for selected stock
stock = yf.Ticker("JNJ")
df = stock.history(period="1y")

In [ ]:
#df.head()

In [ ]:
# Create a moving average function to add on the stock graph
def moving_average(df, window):
  return df.rolling(window=window).mean()

In [ ]:
# function to make graph for stock closing price
def stock_close(df):
    # Create traces
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df.index, y=df['Close'],
                      mode='lines',
                      name='Close'))
  fig.add_trace(go.Scatter(x=df.index, y=moving_average(df['Close'], 10),
                      mode='lines',
                      name='10 Day MA'))
  fig.add_trace(go.Scatter(x=df.index, y=moving_average(df['Close'], 50),
                      mode='lines',
                      name='50 Day MA'))
  return fig.show()

In [ ]:
# Plot volume of the stock
def volume(df):
  fig =px.bar(x=df.index, y=df['Volume'])
  return fig.show()

In [ ]:
# Function to calculate RSI
def rsi(df, alpha):
  df['Change'] = df['Close'].diff()
  df['Gain'] = df['Change'].apply(lambda x: x if x > 0 else 0)
  df['Loss'] = df['Change'].apply(lambda x: abs(x) if x < 0 else 0)
  df['AvgGain'] = df['Gain'].ewm(alpha).mean()
  df['AvgLoss'] = df['Loss'].ewm(alpha).mean()
  df['RS'] = df['AvgGain'] / df['AvgLoss']
  df['RSI'] = 100 - (100 / (1 + df['RS']))
  return df['RSI']

In [ ]:
# Function to graph RSI results
def graph_rsi(df):
  rsi(df, 9)
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df.index, y=df['RSI'],
                      mode='lines',
                      name='RSI',
                      line=dict(color='red')),)
  return fig.show()

In [ ]:
# Function to calculate stochastic oscillator
def stoch(df, period_k, smooth_k, period_d):
  df['Lowest Low'] = df['Low'].rolling(window=period_k).min()
  df['Highest High'] = df['High'].rolling(window=period_k).max()
  df['%K'] = (df['Close'] - df['Lowest Low']) * 100 / (df['Highest High'] - df['Lowest Low'])
  df['%K Smoothed'] = df['%K'].rolling(window=smooth_k).mean()
  df['%D'] = df['%K Smoothed'].rolling(window=period_d).mean()
  return df[['%K Smoothed', '%D']]

In [ ]:
# Function to graph stochastic oscillator
def graph_stoch(df):
  stoch(df, 14, 3, 3)
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df.index, y=df['%K Smoothed'],
                      mode='lines',
                      name='%K Smoothed',
                      line=dict(color='grey')),)
  fig.add_trace(go.Scatter(x=df.index, y=df['%D'],
                      mode='lines',
                      name='%D',
                      line=dict(color='green')),)
  return fig.show()

In [ ]:
# function for MACD
def macd(df, fast_period, slow_period, signal_period):
  df['EMA Fast'] = df['Close'].ewm(span=fast_period, adjust=False).mean()
  df['EMA Slow'] = df['Close'].ewm(span=slow_period, adjust=False).mean()
  df['MACD'] = df['EMA Fast'] - df['EMA Slow']
  df['Signal Line'] = df['MACD'].ewm(span=signal_period, adjust=False).mean()
  df['Histogram'] = df['MACD'] - df['Signal Line']
  return df[['MACD', 'Signal Line', 'Histogram']]


In [ ]:
# Function to return MACD graph
def graph_macd(df, fp, sp, sgp):
  macd(df, fp, sp, sgp)
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df.index, y=df['MACD'],
                      mode='lines',
                      name='MACD',
                      line=dict(color='red')),)
  fig.add_trace(go.Scatter(x=df.index, y=df['Signal Line'],
                      mode='lines',
                      name='Signal Line',
                      line=dict(color='green')),)
  fig.add_trace(go.Bar(x=df.index, y=df['Histogram']))

  return fig.show()

In [ ]:
stock_close(df)

In [ ]:
volume(df)

In [ ]:
graph_rsi(df)

In [ ]:
graph_stoch(df)

In [ ]:
graph_macd(df, 12, 26, 9)